<a href="https://colab.research.google.com/github/sanjana-joshi14/Ecommerce-Analysis/blob/main/RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**MERGING DATA**

In [ ]:
customer_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_customers_dataset.csv')
products_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_products_dataset.csv')
orders_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_orders_dataset.csv')
items_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_order_items_dataset.csv')
cat_names_df = pd.read_csv('/content/drive/MyDrive/Kaggle/product_category_name_translation.csv')
payments_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_order_payments_dataset.csv')
geos_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_geolocation_dataset.csv')
sellers_df = pd.read_csv('/content/drive/MyDrive/Kaggle/olist_sellers_dataset.csv')

In [ ]:
full_df = orders_df.merge(payments_df, on='order_id')\
.merge(items_df, on='order_id')\
.merge(customer_df, on='customer_id')\
.merge(products_df, on='product_id')\
.merge(sellers_df, on='seller_id')

In [ ]:
#PRE REMOVING DUPLICATES
full_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state'],
      dtype='object')

In [ ]:
full_df = full_df.drop_duplicates()

In [ ]:
def df_summary(df):
  summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
  summary = summary.reset_index()
  summary['Column'] = summary['index']
  summary = summary[['Column','dtypes']]
  summary['# unique'] = df.nunique().values
  summary['# missing'] = df.isnull().sum().values
  summary['Example'] = df.loc[0].values

  return summary

In [ ]:
df_summary(full_df)

In [ ]:
full_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state'],
      dtype='object')

In [ ]:
for col in ['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'shipping_limit_date']:
    full_df[col] = pd.to_datetime(full_df[col], errors='coerce')

**DEFINE PERCENTILE FOR NUMERIC COLUMNS**

In [ ]:
full_df.dtypes

In [ ]:
#percentile summary of the numeric and object columns
print(full_df.describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]))
#full_df.describe(include=['number'])

**RFM ANALYSIS**

In [ ]:
#RFM
#full_df['order_purchase_timestamp'] = pd.to_datetime(full_df['order_purchase_timestamp'])
#ref_date = full_df['order_purchase_timestamp'].max() + pd.DateOffset(days=1)

#Recency
#rfm = full_df.groupby(['customer_unique_id','product_category_name']).agg({
#    'order_purchase_timestamp': lambda x: (ref_date - x.max()).days,
#    'order_id': 'count',
#    'payment_value': 'sum'
#  }).rename(columns={
#    'order_purchase_timestamp': 'recency',
#    'order_id': 'frequency'
#})

rfm = rfm.sort_values('frequency', ascending=False)

rfm.head()

,,recency,frequency,payment_value
customer_unique_id,product_category_name,,,
9a736b248f67d166d2fbb006bcb877c3,cama_mesa_banho,391,42,322.64
6fbc7cdadbb522125f4b27ae9dee4060,moveis_escritorio,345,38,411.48
f9ae226291893fda10af7965268fb7f6,ferramentas_jardim,541,35,946.47
8af7ac63b2efbcbd88e5b11505e8098a,ferramentas_jardim,501,29,457.99
569aa12b73b5f7edeaa6f2a01603e381,cama_mesa_banho,453,26,62.68


In [ ]:
# Drop the specified columns
rfm = rfm.drop(columns=['customer_id'])


**OUTLIER ANALYSIS FOR RFM**

In [ ]:
#Assigning Quintile Score
#Recency: Lower values are better (recent activity).
#Frequency and Payment: Higher values are better.

# Define scoring function
def assign_rfm_scores(df, column, ascending=True):

    # Calculate quantiles and drop duplicates
    quantiles = pd.qcut(df[column], 5, duplicates='drop', retbins=False)

     # Determine the number of unique bins
    num_bins = quantiles.cat.categories.size

    # Generate labels based on the actual number of bins
    labels = range(num_bins, 0, -1) if ascending else range(1, num_bins + 1)

    # Apply qcut with the correct number of labels
    return pd.qcut(df[column], 5, labels=labels, duplicates='drop')

# Assign scores
rfm['R_Score'] = assign_rfm_scores(rfm, 'recency', ascending=True)
rfm['F_Score'] = assign_rfm_scores(rfm, 'frequency', ascending=False)
rfm['M_Score'] = assign_rfm_scores(rfm, 'payment_value', ascending=False)


In [ ]:
# Calculate an overall RFM score
rfm['rfm-combined'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

# Combine RFM scores into a single string
rfm['rfm-combined-str'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

In [ ]:
rfm.rename(columns={'rfm-combined': 'rfm_combined'}, inplace=True)

In [ ]:
rfm

In [ ]:
#define rfm percentile
print(rfm.describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]))

**SEGMENTING CUSTOMERS**

In [ ]:
#based on rfm-comnbine score, defining score to form segments is 7

def segments(rfm_combined):
    if rfm_combined >= 10:
       return 'Winners'
    elif rfm_combined <= 8:
       return 'Loyalist'
    elif 5<= rfm_combined >= 7:
       return 'Potential Loyalists'
    else:
       return 'At Risk'

In [ ]:
rfm['Segment'] = rfm['rfm_combined'].apply(segments)

**VISUALIZE SEGMENTS**

In [ ]:
from typing import Sized
from __future__ import annotations
segment_counts = rfm['Segment'].value_counts()

#pie chart
segment_counts.plot(kind='pie',ylabel='Segment',
                    autopct=lambda pct:f'{pct:.1f}% ({int(pct/100.*segment_counts.sum())})')

In [ ]:
#bar chart
barplot = segment_counts.plot(kind='bar',ylabel='Segment')
for p in barplot.patches:
    barplot.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')
barplot.set_xticklabels(barplot.get_xticklabels(), rotation=0)

**MERGING WITH OG DF TO IDENTIFY ISSUES**

In [ ]:
# Merge the two dataframes
#merged_df = pd.merge(full_df, rfm[['rfm_combined', 'Segment']], on='customer_unique_id', how='left')

merged_df.dtypes

**IMPORT DATA TO BIGUERY**

In [ ]:
#Importing main DB as MERGED

# Import necessary libraries
from google.cloud import bigquery
from google.oauth2 import service_account # Import for explicit authentication

# Construct a BigQuery client object.
# Explicitly provide credentials using a service account key file
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/MyDrive/p1-ecomm-f5ffde5adbe7.json'  # Replace with the actual path
)
client = bigquery.Client(credentials=credentials)


# Define your BigQuery dataset and table.
project_id = "p1-ecomm"
dataset_id = "T1_ECOMM"  # Replace with your dataset ID
table_id = "MERGED"     # Replace with your table ID
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)


# Convert the pandas DataFrame to a BigQuery table.
# Replace 'full_df' with the actual variable name containing your dataframe
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE #replace with WRITE_APPEND if you want to add data
job = client.load_table_from_dataframe(
    full_df, table_ref, job_config=job_config
)
job.result()  # Wait for the job to complete.

print(f"Loaded {full_df.shape[0]} rows into BigQuery table {table_id}")